# **Taller:** Clasificador Cuadrático

***Matemáticas para Machine Learning.***

**Semana 3 - Práctica Calificada -** Probabilidad I

**Profesor:** *Fernando Lozano* - **Autor Notebook:** *Nicolás Lopez / César Garrido*

# Introducción

## Descripción

El presente *jupyter notebook* contine todo el material para el desarrollo del Taller de la Semana 3 del curso ***Matemáticas para Machine Learning***. 
En este taller usted tendrá como reto implementar un clásificador cuadrático (QDA) dentro de un problema de clasificación práctico.

**Objetivos de Aprendizaje:**

*   Repasar los conceptos básicos de probabilidad (como probabilidad condicional, Bayes, a priori, a posteriori, etc.) y la importancia de estos, dentro del contexto de un problema práctico de clasificación.
*   Analizar una muestra de datos reales.
*   Entender e implemntar el modelo de Clasificador Cuadrático (QDA).
*   Analizar los alcances y limitaciones de la clasificación con el modelo de QDA.

## Teoria

$$ \textrm{QDA} $$

Describir como funciona, par acuaciones de de donde viene, que signifca cada variable.

https://towardsdatascience.com/quadratic-discriminant-analysis-ae55d8a8148a

**Clasificador cuadrático:**

El clasificador cuadrático o análisis de discriminante cuadrático (QDA por sus siglas en inglés) es un modelo generativo, sí a pesar de que el nombre diga lo contrario, que utiliza las distribuciones conjuntas de probabilidad de los datos para resolver problemas de clasificación.


QDA parte del supuesto que las densidades de probabilidad condicional de todas las clases están distribuidas de forma normal (gaussiana). Lo que significa que para un dataset de datos de entrenamiento ($x$) con etiquetas ($y$):

\begin{equation}
P(x | y=c, \mu_c, \Sigma_c) = \mathcal{N}(x|\mu_c, \Sigma_c)
\end{equation}

Donde $\mu_c$ es la media estadistica de la clase $c$ y $\Sigma_c$ es la matriz de covariancia correspondiente para la clase $c$.

**¿Cómo clasifica?**

Ahora bien, con el ya conocido y trabajado teorema de bayes se puede calcular lo que se conoce como la probabilidad *a posteriori*. Es decir, la probabilidad de que $x$ (la nueva observación) pertenezca a la clase $c$, dados la distribución de los datos con parametros $\mu_c$ y $\Sigma_c$:

\begin{equation}
P(y=c|x, \mu_c, \Sigma_c) = \frac{P(x|y=c, \mu_c, \Sigma_c) P(y=c)}{ \sum_{k=1}^K P(x|y=c, \mu_k, \Sigma_k) P(y=k)}
\end{equation}

Donde se tiene la densidad de probabilidad condicional de la clase $c$ (modelada por una gaussiana), ponderada por la probbilidad a priori de la clase ($P(y=c)$), normalizado por la suma de probabilidades de todas las clases.

Así las cosas, se clasifica una observación ($x$) en una clase ($c$) dada la máxima probabilidad a posteriori:


\begin{equation}
\hat{h}(x) = argmax_c P(y=c|x, \mu_c, \Sigma_c)
\end{equation}

Adicionalmente en el analísis de discriminante cuadrático (QDA), se parte del hecho que se tienen distribuciones normales, con medias y covarianzas no necesariamente iguales. Ademas se utiliza siguiente función discriminatne para una de estas distribuciones $k$:

$$ \delta_k(\mathbf{x}) = - \frac{1}{2} \log |\Sigma_k| - \frac{1}{2} (\mathbf{x} - \mu_k)^T \Sigma_k^{-1}(\mathbf{x} - \mu_k) + \log \pi_k $$

En donde $\Sigma_k$ y $\mu_k$ son la matriz de covarianza y la media de la distribución $k$, y $\pi_k$ es la probabilidad a priori para la clase $k$. Una vez hemos identificado este valor de discriminante podemos observar que las fronteras de decisión para este método estarán dadas por puntos que satisfacen:

$$ \frac{1}{2} \log |\Sigma_l| +  \frac{1}{2} (\mathbf{x} - \mu_l)^T \Sigma_l^{-1}(\mathbf{x} - \mu_l) - \log \pi_l = \frac{1}{2} \log |\Sigma_k| +  \frac{1}{2} (\mathbf{x} - \mu_k)^T \Sigma_k^{-1}(\mathbf{x} - \mu_k) - \log \pi_k $$

para $l,k$ distribuciones distintas.


## Metodología

Para desarrollar el taller usted deberá editar las celdas de código dispuestas para esto. Estas estarán marcadas con el siguiente comentario:

```python
# =====================================================
# COMPLETAR ===========================================
# 

# =====================================================
```

Edite o complete el códgio dentro de estas lineas de comentarios. Dentro de estos comentarios encontrará indicaciónes de lo que debe hacer, así como algunas de las variables que debe utilizar o calcular (puede que estas tengan ya una estructura para llenar o esten solo igualadas a None, complete la asignación).

# Problema Práctico

El problema práctico que se desea resolver en este taller es el de **clasificación de pulsares**. Estos astros son un tipo de estrella de neutrones que emite radiacion detectable en la Tierra. Estas estrellas son de gran interes en distintas áreas de la física y la astronomía, pero requieren de un análisis detallado de los datos para ser detectadas. Es por esto último que en las decadas recientes se han empleado técnicas de *Machine Learning* para desarrollar modelos que las detecten. 

Para esto usted va a trabjar con un subset de datos del dataset HTRU2. Este contiene únicamente dos mediciones astronómicas relevantes (2 variables de entrada) con las que usted va a tratar de detectar posibles pulsares. Este es un problema binario de dos clases, los datos corresponden a un pulsar o no sencillamente no corresponden a un pulsar.

## Inicialización

In [ ]:
# Librerias principales
import os
import cv2
import numpy as np
import pandas as pd

# Sklearn
from sklearn.datasets import load_iris
from sklearn.decomposition import TruncatedSVD, PCA

# Visualización
import plotly.express as px
import matplotlib.pyplot as plt

## Datos


A continuación usted encontrará un primer set de datos para dos de las variables descritas anteriormente con su respectiva etiqueta.

**Columna Pulsar:** 
*  0 = Datos NO corresponden a un pulsar
*  1 = Datos corresponden a un pulsar. 

In [ ]:
# Carga e imprime primeros datos de la muestra
datos_muestra = pd.read_csv("first_data.csv")
datos_muestra.head(5)

In [ ]:
# Gráfica datos de la muestra
datos_muestra["Pulsar"] = datos_muestra["Pulsar"].astype(str)
fig = px.scatter(datos_muestra, title="Distribución Inicial de los datos", x="Exceso de curtosis del perfil integrado", y="Exceso de curtosis de la curva DM-SNR", color="Pulsar")
fig.show()

### Inspección de los datos

Ahora bien, a partir de esta muestra usted debe calcular los parámetros que describen estos datos, es decir:

*   Media
*   Varianza

In [ ]:
# =====================================================
# COMPLETAR ===========================================
# -
# AYUDA:
distribucion_datos = 0

media_pulsar = 0
varianza_pulsar = np.zeros((2,2))

media_no_pulsar = 0
varianza_no_pulsar = np.zeros((2,2))
# =====================================================

## Completar Algoritmo

A continuación, se presenta una implementación del algoritmo de discriminante lineal. En esta implementación se sabe que las probabilidades de que sucedan los distintos están balanceadas o son desconocidas, por lo que las probabilidades a priori no son tenidas en cuenta. Complete la implementación para que el algoritmo funcione

### QDA sin información previa

In [ ]:
class QDA0:
    def fit(self, X, t):

        # Inicializa
        self.means = dict()
        self.covs = dict()
        
        # Identifica clases únicas
        self.classes = np.unique(t)

        # Obtiene información relevante a cada clase
        for c in self.classes:
            # =====================================================
            # COMPLETAR ===========================================
            # -
            # AYUDA:
            self.means[c] = None
            self.covs[c] = None
            # =====================================================
            

    def predict(self, X):
        preds = list()
        for x in X:
            likelihoods = list()
            for c in self.classes:
                # =====================================================
                # COMPLETAR ===========================================
                # -
                # AYUDA:
                inv_cov = None
                inv_cov_det = None
                diff = None
                likelihood = None
                likelihoods.append(likelihood)
                # =====================================================
            
            # Selecciona una predicción  
            pred = self.classes[np.argmax(likelihoods)]
            preds.append(pred)
            
        return np.array(preds)

### QDA completo

Ahora complete QDA suponiento que tiene acceso a información a priori respecto a la distribución de las muestras.

In [ ]:
class QDA:
    def fit(self, X, t):

        # Inicializa
        self.priors = dict()
        self.means = dict()
        self.covs = dict()
        
        # Identifica clases únicas
        self.classes = np.unique(t)

        # Obtiene información relevante a cada clase
        for c in self.classes:
            # =====================================================
            # COMPLETAR ===========================================
            # -
            # AYUDA:
            self.means[c] = None
            self.covs[c] = None
            # =====================================================
            

    def predict(self, X):
        preds = list()
        for x in X:
            posts = list()
            for c in self.classes:
                # =====================================================
                # COMPLETAR ===========================================
                # -
                # AYUDA:
                inv_cov = None
                inv_cov_det = None
                diff = None
                likelihood = None
                post = None
                posts.append(post)
                # =====================================================
            
            # Selecciona una predicción  
            pred = self.classes[np.argmax(posts)]
            preds.append(pred)

        return np.array(preds)

In [ ]:
class QDA_sol:
    def fit(self, X, t):
        self.priors = dict()
        self.means = dict()
        self.covs = dict()
        
        # Identifica clases únicas
        self.classes = np.unique(t)

        # Obtiene información relevante a cada clase
        for c in self.classes:
            X_c = X[t == c]
            self.priors[c] = X_c.shape[0] / X.shape[0]
            self.means[c] = np.mean(X_c, axis=0)
            self.covs[c] = np.cov(X_c, rowvar=False)
            

    def predict(self, X):
        preds = list()
        for x in X:
            posts = list()
            for c in self.classes:
                prior = np.log(self.priors[c])
                inv_cov = np.linalg.inv(self.covs[c])
                inv_cov_det = np.linalg.det(inv_cov)
                diff = x-self.means[c]
                likelihood = 0.5*np.log(inv_cov_det) - 0.5*diff.T @ inv_cov @ diff
                post = prior + likelihood
                posts.append(post)
            pred = self.classes[np.argmax(posts)]
            preds.append(pred)
        return np.array(preds)

## Implementación y Visualización.

### Métricas

Utilice medidas de Precisión (1's identificados correctamente / 1's totales) , Exhaustividad ( 1's identificados correctamente / 1's Predichos ) y Exactitud ( 1's y 0's identificados correctamente / Total de datos) para comparar ambas implementaciones de QDA. Cree una función que a partir de un conjunto de datos y una implementación de QDA obtenga estos 

In [ ]:
def obtener_metricas(pred, real, target=1):
    # =====================================================
    # COMPLETAR ===========================================
    # -
    # AYUDA:
    acc = 0
    prec = 0
    recc = 0
    # =====================================================
    return acc, prec, recc

### Implementación sobre muestra balanceada

Utilice la implementación de QDA sin inrformación previa para completar la siguiente sección.

Pruebe la clasificación con la primera muestra de datos.

In [ ]:
# Prueba algoritmo


#### Visualización

In [ ]:
# Grafica


### Implementación sobre todos los datos

Considere que ahora tiene acceso a todo el conjunto de datos tomados durante el año.



In [ ]:
# Carga e imprime primeros datos de la muestra
datos_muestra = pd.read_csv("complete_data.csv")
datos_muestra.head(5)

Inspeccione nuevamente ahora estos datos, calcule los parametros y ponga a prueba nuevamente el algoritmo implementado en el literal anterior.

¿Qué cambios observa en la inspección de los parámetros? ¿Es sustancial el cambio en la clasificación? Explique su respuesta.

In [ ]:
# =====================================================
# COMPLETAR ===========================================
# -
# AYUDA:
distribucion_datos = 0

media_pulsar = 0
varianza_pulsar = np.zeros((2,2))

media_no_pulsar = 0
varianza_no_pulsar = np.zeros((2,2))
# =====================================================

#### Visualización

In [ ]:
# Grafica

